In [ ]:
from GENIE3 import *

In [ ]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import SAUCIE
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
from tqdm import tqdm

sys.path.append(os.getcwd())

In [ ]:
sys.path

In [ ]:
# y = np.transpose(np.load('./previous_imputations/DS6_45.npy'))
y_test = np.transpose(np.load('../SERGIO/imputation_data/DS3/DS6_45.npy'))

In [ ]:
x = np.transpose(np.load('../SERGIO/imputation_data/DS3/DS6_clean.npy'))

In [ ]:
# y_hat_saucie = np.load('./baselines/SAUCIE/yhat_saucie.npy')

In [ ]:
# y_magic_2 = np.load('./baselines/MAGIC/yhat_MAGIC_t_2.npy')
# y_magic_7 = np.load('./baselines/MAGIC/yhat_MAGIC_t_7.npy')
# y_magic_default = np.load('./baselines/MAGIC/yhat_MAGIC_t_auto.npy')

In [ ]:
target_file = '../SERGIO/data_sets/temp/De-noised_1200G_9T_300cPerT_6_DS3/Interaction_cID_6.txt'

In [ ]:
# magic_df = pd.DataFrame(y_magic_2)

In [ ]:
file = open('../SERGIO/data_sets/temp/De-noised_1200G_9T_300cPerT_6_DS3/Interaction_cID_6.txt', 'r')
lines = file.readlines()
regulators = []
for row in lines:
    row = row.split(',')
    num_regs_row = int(float(row[1]))
    if num_regs_row != 0:
        for i in range(2, 2 + num_regs_row):
            regulators.append(str(int(float(row[i]))))
file.close()
regulators = list(set(regulators))

In [ ]:
master_regulators = pd.read_table('../SERGIO/data_sets/temp/De-noised_1200G_9T_300cPerT_6_DS3/Regs_cID_6.txt', header=None, sep=',')
master_regulators = master_regulators[0].astype(int).astype(str).tolist()

In [ ]:
true_regs = [x for x in regulators if x not in master_regulators]

In [ ]:
other_regs = pd.read_csv('../SERGIO/data_sets/temp/De-noised_1200G_9T_300cPerT_6_DS3/gt_GRN.csv', header=None)
other_regs = list(set(other_regs[0].values.astype(str).tolist()))
hmm = [x for x in other_regs if x not in master_regulators]
print(len(regulators), len(other_regs), len(master_regulators), len(true_regs), len(hmm))

In [ ]:
print(len(regulators), len(master_regulators), len(true_regs))

In [ ]:
VIM = GENIE3(x, nthreads=40, ntrees=80, regulators='all', gene_names=[str(s) for s in range(x.shape[1])])
# VIM_1 = GENIE3(y_magic_2,nthreads=12, ntrees=100, regulators=regulators, gene_names=[str(s) for s in range(y_magic_2.shape[1])])
# VIM_2 = GENIE3(y_magic_2,nthreads=12, ntrees=100, regulators=master_regulators, gene_names=[str(s) for s in range(y.shape[1])])
# VIM_3 = GENIE3(y_magic_2,nthreads=12, ntrees=100, regulators=true_regs, gene_names=[str(s) for s in range(y.shape[1])])

In [ ]:
from sklearn.metrics import roc_auc_score
# for i in range(4):
v = VIM
# if i == 1:
#     print("all regulators")
#     v = VIM_1
# if i == 2:
#     print("master regulators")
#     v = VIM_2
# if i == 3:
#     print("'true' regulators")
#     v = VIM_3
# if i == 0:
#     print("baseline")
gt = np.zeros_like(v)
f = open(target_file,'r')
Lines = f.readlines()
f.close()
for j in range(len(Lines)):
    line = Lines[j]
    line_list = line.split(',')
    target_index = int(float(line_list[0]))
    num_regs = int(float(line_list[1]))
    for i in range(num_regs):
        try:
            reg_index = int(float(line_list[i+2]))
            gt[reg_index,target_index] = 1 
        except:
            continue
print(roc_auc_score(gt.flatten(),v.flatten()))  

Saucie imputed data performance

In [ ]:
y_hat_scscope = np.load('../scScope/yhat_scScope.npy')

In [ ]:
VIM = GENIE3(y_hat_scscope,nthreads=12,ntrees=100)

scScope imputed data performance

In [ ]:
roc_auc_score(gt.flatten(),VIM.flatten())

In [ ]:
VIM = GENIE3(x,nthreads=12,ntrees=100)

Clean data performance

In [ ]:
roc_auc_score(gt.flatten(),VIM.flatten())

In [ ]:
VIM = GENIE3(y,nthreads=12,ntrees=100)

Noisy data performance

In [ ]:
roc_auc_score(gt.flatten(),VIM.flatten())

## DeepImpute

In [ ]:
y_hat_deepimp = np.load('./baselines/deepimpute/yhat_deepimp.npy')

In [ ]:
VIM = GENIE3(y_hat_deepimp,nthreads=12,ntrees=100)

In [ ]:
roc_auc_score(gt.flatten(),VIM.flatten())

## MAGIC

In [ ]:
y_hat_magic = np.load('../MAGIC/yhat_MAGIC.npy')

In [ ]:
VIM = GENIE3(y_hat_magic, nthreads=12, ntrees=100)

In [ ]:
roc_auc_score(gt.flatten(), VIM.flatten())